# Using Automated Machine Learning

There are many kinds of machine learning algorithm that you can use to train a model, and sometimes it's not easy to determine the most effective algorithm for your particular data and prediction requirements. Additionally, you can significantly affect the predictive performance of a model by preprocessing the training data, using techniques such as normalization, missing feature imputation, and others. In your quest to find the *best* model for your requirements, you may need to try many combinations of algorithms and preprocessing transformations; which takes a lot of time and compute resources.

Azure Machine Learning enables you to automate the comparison of models trained using different algorithms and preprocessing options. You can use the visual interface in [Azure Machine Learning studio](https://ml/azure.com) or the SDK to leverage this capability. The SDK gives you greater control over the settings for the automated machine learning experiment, but the visual interface is easier to use. In this lab, you'll explore automated machine learning using the SDK.

## Before You Start

Before you start this lab, ensure that you have completed the *Create an Azure Machine Learning Workspace* and *Create a Compute Instance* tasks in [Lab 1: Getting Started with Azure Machine Learning](./labdocs/Lab01.md). Then open this notebook in Jupyter on your Compute Instance.

## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK.

> **Note**: If you do not have a current authenticated session with your Azure subscription, you'll be prompted to authenticate. Follow the instructions to authenticate using the code provided.

In [8]:
import azureml.core
from azureml import widgets, core
from azureml.train import automl
from azureml.core import experiment


In [2]:
ws = azureml.core.Workspace.from_config()
print(f'Ready to use Azure ML {azureml.core.VERSION} to work with {ws.name}')

Ready to use Azure ML 1.12.0 to work with workspace


## Prepare Data for Automated Machine Learning

You don't need to create a training script for automated machine learning, but you do need to load the training data. In this case, you'll create a dataset containing details of diabetes patients (just as you did in previous labs), and then split this into two datasets: one for training, and another for model validation.

In [3]:
default_ds = ws.get_default_datastore()

print('Dataset already registered.')

diabetes_ds = ws.datasets.get("diabetes dataset")
train_ds, test_ds = diabetes_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")


Dataset already registered.
Data ready!


## Configure Automated Machine Learning

Now you're ready to configure the automated machine learning experiment. To do this, you'll need a run configuration that includes the required packages for the experiment environment, and a set of configuration settings that specifies how many combinations to try, which metric to use when evaluating models, and so on.

> **Note**: In this example, you'll run the automated machine learning experiment on local compute to avoid waiting for a cluster to start. This will cause each iteration (child-run) to run serially rather than in parallel. For this reason, we're restricting the experiment to 6 iterations to reduce the amount of time taken. In reality, you'd likely try many more iterations on a compute cluster.

In [4]:
automl_config = automl.AutoMLConfig(
    'classification',
    iterations=6,
    primary_metric = 'AUC_weighted',
    compute_target='local',
    training_data = train_ds,
    validation_data = test_ds,
    label_column_name='Diabetic',
)

print("Ready for Auto ML run.")


Ready for Auto ML run.


## Run an Automated Machine Learning Experiment

OK, you're ready to go. Let's run the automated machine learning experiment.

In [5]:
print('Submitting Auto ML experiment...')
automl_experiment = experiment.Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
widgets.RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)


Submitting Auto ML experiment...


WARNING - Running on local machine. Note that local runs always run synchronously if not using Docker containers, even if you use the parameter 'show_output=False'


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_3ae352c3-1edc-4b9d-b032-323f54bd8755',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-08-21T22:59:44.300626Z',
 'endTimeUtc': '2020-08-21T23:01:39.435909Z',
 'properties': {'num_iterations': '6',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'local',
  'RawAMLSettingsString': None,
  'AMLSettingsJsonString': '{"path":null,"name":"diabetes_automl","subscription_id":"84170def-2683-47c0-91ed-1f34057afd69","resource_group":"resources","workspace_name":"workspace","region":"brazilsouth","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"iterations":6,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":

## Determine the Best Performing Model

When the experiment has completed, view the output in the widget, and click the run that produced the best result to see its details.
Then click the link to view the experiment details in the Azure portal and view the overall experiment details before viewing the details for the individual run that produced the best result. There's lots of information here about the performance of the model generated.

Let's get the best run and the model that it produced.

In [6]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: diabetes_automl,
Id: AutoML_3ae352c3-1edc-4b9d-b032-323f54bd8755_0,
Type: None,
Status: Completed)
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, force_text_dnn=None,
        is_cross_validation=None, is_onnx_compatible=None, logger=None,
        obser...    silent=True, subsample=1.0, subsample_for_bin=200000,
          subsample_freq=0, verbose=-10))])
precision_score_macro 0.9467885716011675
average_precision_score_micro 0.9918359683181054
average_precision_score_macro 0.9888863976623419
recall_score_micro 0.9520809898762654
AUC_micro 0.9916008305485999
AUC_weighted 0.9904812577250306
recall_score_macro 0.9451102807249344
AUC_macro 0.9904812577250306
recall_score_weighted 0.9520809898762654
weighted_accuracy 0.9576485145517856
precision_score_weighted 0.9520038306966419
matthews_correlation 0.891

Automated machine learning includes the option to try preprocessing the data, which is accomplished through the use of [Scikit-Learn transformation pipelines](https://scikit-learn.org/stable/modules/compose.html#combining-estimators) (not to be confused with Azure Machine Learning pipelines!). These produce models that include steps to transform the data before inferencing. You can view the steps in a model like this:

In [7]:
for step in fitted_model.named_steps:
    print(step)

datatransformer
MaxAbsScaler
LightGBMClassifier


Finally, having found the best performing model, you can register it.

In [9]:
best_run.register_model(
    model_path='outputs/model.pkl', model_name='diabetes_model_automl',
    tags={'Training context':'Auto ML'},
    properties={
        'AUC': best_run_metrics['AUC_weighted'],
        'Accuracy': best_run_metrics['accuracy']
    }
)

for model in core.Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')


diabetes_model_automl version: 1
	 Training context : Auto ML
	 AUC : 0.9904812577250306
	 Accuracy : 0.9520809898762654


diabetes_model version: 11
	 Training context : Hyperdrive
	 AUC : 0.856969468262725
	 Accuracy : 0.7891111111111111


diabetes_model version: 10
	 Training context : Inline Training
	 AUC : 0.8770884123588237
	 Accuracy : 0.8893333333333333


diabetes_model version: 9
	 Training context : Inline Training
	 AUC : 0.8778421812030448
	 Accuracy : 0.8903333333333333


diabetes_model version: 8
	 Training context : Pipeline


diabetes_model version: 7
	 Training context : Pipeline


diabetes_model version: 6
	 Training context : Pipeline


diabetes_model version: 5
	 Training context : Parameterized SKLearn Estimator
	 AUC : 0.8483904671874223
	 Accuracy : 0.7736666666666666


diabetes_model version: 4
	 Training context : Parameterized SKLearn Estimator
	 AUC : 0.8483904671874223
	 Accuracy : 0.7736666666666666


diabetes_model version: 3
	 Training context : Estimato

### More Information

For more information Automated machine Learning, see the [Azure ML documentation](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train).

## Clean Up

If you've finished exploring, you can close this notebook and shut down your Compute Instance.